# Get Ford dealer locations

#### Load Python tools and Jupyter config

In [1]:
import us
import json
import requests
import pandas as pd
import jupyter_black
import altair as alt
import geopandas as gpd
from bs4 import BeautifulSoup
from vega_datasets import data
from tqdm.notebook import tqdm, trange

In [2]:
jupyter_black.load()
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [3]:
place = "ford"
place_formal = "Ford"
color = "#0068d0"
latitude = "39.106667"
longitude = "-94.676392"

## Scrape

#### Headers for the request

In [88]:
import requests

cookies = {
    "__cf_bm": "ZbMqGYe7BA3C6Yjv80dOcu441onwbmuFHP60taJvi3o-1709687671-1.0.1.1-K7_KSx9SrdSLLiwkz6tcIhS7Pc0RhqVTXj1nnXuj7p6gNWZvzKqLzYHLh3jYsGAcP4kSPS_0iRFiaRoNVQ4AOw",
    "_dy_ses_load_seq": "83366%3A1709687671773",
    "_dy_csc_ses": "t",
    "_dy_c_exps": "",
    "_dycnst": "dg",
    "_dyid": "-6480679612969144075",
    "_dyjsession": "6506153fad19e9b7f7063b542dfa1ca9",
    "dy_fs_page": "www.forever21.com%2F%3Futm_source%3Dgoogle%26utm_medium%3Dcpc%26utm_campaign%3Dfpmx_f21_brand_core_search_exact%26utm_id%3D71700000098624925%26gad_source%3D1%26gclid%3Dcjwkcaiaopuvbhbceiwam8jamvb3sjwh0h9jwux5jax31tejlclpnyf2zt0fphp4_zeszyoj885khhocvseqavd_bwe%26gclsrc%3Daw.ds",
    "_dy_lu_ses": "6506153fad19e9b7f7063b542dfa1ca9%3A1709687671951",
    "_dycst": "dk.m.c.ms.",
    "_dy_geo": "US.NA.US_CA.US_CA_Burbank",
    "_dy_df_geo": "United%20States.California.Burbank",
    "_dy_toffset": "0",
    "_dy_soct": "482065.880923.1709687671*829696.1626594.1709687671*829697.1626604.1709687671*829720.1626755.1709687671*829723.1626778.1709687671*829726.1626789.1709687671",
    "_gid": "GA1.2.225391541.1709687673",
    "_gac_UA-233210-1": "1.1709687673.CjwKCAiAopuvBhBCEiwAm8jaMVB3SJWh0H9jwUX5jaX31TeJlclpNYF2zt0FpHp4_ZESZyOj885KhhoCvSEQAvD_BwE",
    "__cq_uuid": "bccGeI7Cz6XAEW1ujacrmRoC0s",
    "__cq_seg": "0~0.00!1~0.00!2~0.00!3~0.00!4~0.00!5~0.00!6~0.00!7~0.00!8~0.00!9~0.00",
    "_gcl_aw": "GCL.1709687674.CjwKCAiAopuvBhBCEiwAm8jaMVB3SJWh0H9jwUX5jaX31TeJlclpNYF2zt0FpHp4_ZESZyOj885KhhoCvSEQAvD_BwE",
    "_gcl_dc": "GCL.1709687674.CjwKCAiAopuvBhBCEiwAm8jaMVB3SJWh0H9jwUX5jaX31TeJlclpNYF2zt0FpHp4_ZESZyOj885KhhoCvSEQAvD_BwE",
    "_gcl_au": "1.1.562972004.1709687674",
    "_scid": "ee637be7-78b1-4857-b396-8eb2ab8f5e18",
    "IR_gbd": "forever21.com",
    "_svsid": "c608766583ed9c253e05db086fe49b59",
    "_derived_epik": "dj0yJnU9UWVlRWJxQVBBRVM2d3I3UTZRRGRfODFaZWMxUHNEOHMmbj03S0lWa3MxMHJDLVVtc2I3SjlIVjRBJm09MSZ0PUFBQUFBR1hudzNvJnJtPTEmcnQ9QUFBQUFHWG53M28mc3A9Mg",
    "_pin_unauth": "dWlkPU5UQXdaRGd3WlRNdE5qRTVNeTAwWWpkaUxUazNPVGt0T0dKak0yTmxOR014WkdZeA",
    "QuantumMetricSessionID": "9eddb19d001be29b80c52f97b303e1c3",
    "QuantumMetricUserID": "9c5f6cbecdaf948f524b42f83fc94783",
    "_tt_enable_cookie": "1",
    "_ttp": "ZtzAKihoUWic-uG6MnNFixgCheQ",
    "btIdentify": "a4a62258-bf52-427e-c095-d40c29ede0bb",
    "_bts": "5339c4c6-8cdf-47ea-e334-a3a0369c3668",
    "IR_PI": "b18bd429-c761-11ee-ab38-433b97cf5fdb%7C1709774075035",
    "mp_forever21_mixpanel": "%7B%22distinct_id%22%3A%20%2218e115397247ae-0bb86616eafad9-1d525637-121cf0-18e1153972524f1%22%2C%22bc_persist_updated%22%3A%201709687674662%2C%22g_search_engine%22%3A%20%22google%22%7D",
    "_bti": "%7B%22app_id%22%3A%22forever-21%22%2C%22bsin%22%3A%220X25XRQFFx8SH%2Fde5c%2B21NApnMhMUn5aHeoJqcvLxxPcyYSGKmxlrxzxeY3AIXj%2FM4YiNal%2BDxKG3dYdgkiGIg%3D%3D%22%2C%22is_identified%22%3Afalse%7D",
    "_sctr": "1%7C1709625600000",
    "gtm_sess_ref": "https://www.forever21.com/",
    "gtm_sess_ts": "2024-03-06T01:14:55.750Z",
    "_pin_unauth": "dWlkPU5UQXdaRGd3WlRNdE5qRTVNeTAwWWpkaUxUazNPVGt0T0dKak0yTmxOR014WkdZeA",
    "__attentive_id": "493aea821c524b459d4df372f7559f3e",
    "_attn_": "eyJ1Ijoie1wiY29cIjoxNzA5Njg3Njk2MjQ5LFwidW9cIjoxNzA5Njg3Njk2MjQ5LFwibWFcIjoyMTkwMCxcImluXCI6ZmFsc2UsXCJ2YWxcIjpcIjQ5M2FlYTgyMWM1MjRiNDU5ZDRkZjM3MmY3NTU5ZjNlXCJ9In0=",
    "__attentive_cco": "1709687696250",
    "ga_events": "%5B%7B%22clientId%22%3A%22819671682.1709687672%22%2C%22trackingId%22%3A%22UA-233210-1%22%2C%22name%22%3A%22gtm31%22%7D%2C%7B%22clientId%22%3A%22819671682.1709687672%22%2C%22trackingId%22%3A%22UA-233210-1%22%2C%22name%22%3A%22gtm163%22%7D%5D",
    "__attentive_ss_referrer": "https://www.forever21.com/",
    "__attentive_dv": "1",
    "__attentive_vf": "true",
    "OptanonAlertBoxClosed": "2024-03-06T01:15:01.012Z",
    "__cf_bm": "0My6X8u1LQdKMaXoOMzO06uXoA9nNDtetChP46WRKWg-1709687745-1.0.1.1-BiWoHLv0.gp0TpFU_PrYZDomVnJvfgaKXtxDURgc8wIyVLw8bpqOPxGJHX803M.hiakmKFbIbNKgEpUOR09tfw",
    "OptanonConsent": "isGpcEnabled=0&datestamp=Tue+Mar+05+2024+17%3A15%3A45+GMT-0800+(Pacific+Standard+Time)&version=6.20.0&browserGpcFlag=0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0001%3A1%2CC0002%3A1%2CC0003%3A1%2CC0004%3A1&AwaitingReconsent=false&geolocation=US%3BCA",
    "_ga": "GA1.2.819671682.1709687672",
    "_br_uid_2": "uid%3D4061768976743%3Av%3D11.8%3Ats%3D1709687672776%3Ahc%3D9",
    "_ga_L1T9QR4S4K": "GS1.1.1709687672.1.1.1709687745.60.0.0",
    "_scid_r": "ee637be7-78b1-4857-b396-8eb2ab8f5e18",
    "IR_22842": "1709687745663%7C0%7C1709687745663%7C%7C",
    "_uetsid": "e4c09b70db5611eeb971872178bd2b9d",
    "_uetvid": "e4c0ddb0db5611eeb78117241097dbb8",
    "_derived_epik": "dj0yJnU9M2ZFMENIT3MzRjVLUlQ2QjdIeDNkN3B2TTdQT3FHY3Umbj1pMjJ1TTkxVThmd1o4aS00Ty1IQ0pRJm09MSZ0PUFBQUFBR1hudzhFJnJtPTEmcnQ9QUFBQUFHWG53OEUmc3A9Mg",
    "__attentive_pv": "8",
    "cto_bundle": "pBjn619NS1NvWlJjajJBakdvTndEQmhUQ2s5bTFHdlVociUyRjN6cjBGUWFMS2wzamZTOGU1dnZDN21lS2xYSmZ6anV5byUyQmJQd0hsdUdsQktENU9ZanJ6JTJCWXRZYzdXc3glMkIlMkJhcmJxSks3M1MyTHl1Qmt4SXlpZ2dJYjdUQkhtcUhzS1REWWlrVGhVWEVHcGk1UWJyeWhKRlklMkZ2VWp0STgybmZjMHlCJTJCbU9aRWhmOWJEOHBEdGZYRVFQUzlrbVVWcFdjZDVHeThoJTJCVEhwUko0MDdxdERveVVpUk92N2RhSklxRVU1SDZaZWxaNEJ3MFFmM2ZwOGhnZ09UWHBQZVhENnVaNkRBYk05SUpWZlQ1UHRXT1o3NkxEYWw5TkElM0QlM0Q",
}

headers = {
    "authority": "locations.forever21.com",
    "accept": "application/json",
    "accept-language": "en-US,en;q=0.9,es;q=0.8",
    "referer": "https://locations.forever21.com/index.html?q=90066&qp=90066&l=en",
    "sec-ch-ua": '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"macOS"',
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
}

params = {
    "q": "",
    "qp": "",
    "l": "en",
}

response = requests.get(
    "https://locations.forever21.com/index.html",
    params=params,
    headers=headers,
)

In [89]:
response.json()["response"]["entities"]

[{'derivedData': {'address': {'countryName': 'United States',
    'stateName': 'California'},
   'uber': {'url': 'https://m.uber.com/ul/?action=setPickup&client_id=KXQcwoj2Zb8ymDzKgVgbIaDE5iAE_TAj&dropoff%5Bformatted_address%5D=200%20Inland%20Dr%2C%20null%2C%20San%20Bernardino%2C%20CA&dropoff%5Blatitude%5D=34.08670124988919&dropoff%5Blongitude%5D=-117.29933838615399&dropoff%5Bnickname%5D=Forever%2021&pickup=my_location',
    'wrappedUrl': 'http://a.gotoloc.com/uber/FXckS6yHX2E'}},
  'distance': None,
  'listings': {'googleMyBusiness': {'placeId': 'ChIJp7cbqcus3IARCBbrgeaG5rI',
    'url': 'https://maps.google.com/maps?cid=12891139307974891016'}},
  'profile': {'address': {'city': 'San Bernardino',
    'countryCode': 'US',
    'extraDescription': None,
    'line1': '200 Inland Dr',
    'line2': None,
    'line3': None,
    'postalCode': '92408',
    'region': 'CA',
    'sublocality': None},
   'c_aboutCTA': {'label': 'Visit Forever21.com',
    'link': 'http://www.forever21.com',
    'lin